In [ ]:
# Step 1: Importing all necessary libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Step 2: Load and preprocess the data
# Load the dataset
df = pd.read_csv('/content/Iris.csv')

# Encode target labels into integers
label_encoder = LabelEncoder()
df['species'] = label_encoder.fit_transform(df['species'])  # species → 0,1,2

# Separate features and labels
X = df.drop('species', axis=1).values
y = df['species'].values

# Normalize feature values
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into 80% training and 20% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)

# Step 3: Data Visualization
# Boxplots by species
df_vis = pd.read_csv('iris.csv')
plt.figure(figsize=(12, 8))
for idx, column in enumerate(df_vis.columns[:-1]):
    plt.subplot(2, 2, idx+1)
    sns.boxplot(data=df_vis, x='species', y=column)
    plt.title(f'Boxplot of {column} by species')
plt.tight_layout()
plt.show()

# Optional: Pairplot
sns.pairplot(df_vis, hue='species')
plt.show()

# Step 4: Define the Neural Network Architecture
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)     # 4 features → 16 neurons
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 3)     # 16 neurons → 3 classes

    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.fc2(out)
        return out

# Instantiate the model
model = IrisNet()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 5: Train the model
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

# Number of epochs
epochs = 100

for epoch in range(1, epochs+1):
    # Forward pass and loss
    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    # Backpropagation and optimizer step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Logging every 10 epochs
    if epoch % 10 == 0:
        # Evaluate on training data
        train_pred = torch.argmax(model(X_train), dim=1)
        train_acc = accuracy_score(y_train, train_pred)
        train_loss = criterion(model(X_train), y_train).item()

        # Evaluate on test data
        test_pred = torch.argmax(model(X_test), dim=1)
        test_acc = accuracy_score(y_test, test_pred)
        test_loss = criterion(model(X_test), y_test).item()

        train_losses.append(train_loss)
        test_losses.append(test_loss)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)

        print(f"Epoch {epoch}: Train Loss={train_loss:.4f}, Test Loss={test_loss:.4f}, "
              f"Train Acc={train_acc:.4f}, Test Acc={test_acc:.4f}")

# Step 6: Plotting Loss and Accuracy Graphs
# Plot 1: Training and test loss vs. epochs
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(10, 101, 10), train_losses, label='Train Loss')
plt.plot(range(10, 101, 10), test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs Epochs')
plt.legend()

# Plot 2: Training and test accuracy vs. epochs
plt.subplot(1, 2, 2)
plt.plot(range(10, 101, 10), train_accuracies, label='Train Accuracy')
plt.plot(range(10, 101, 10), test_accuracies, label='Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Epochs')
plt.legend()
plt.tight_layout()
plt.show()

# Plot 3: Confusion matrix
final_preds = torch.argmax(model(X_test), dim=1)
cm = confusion_matrix(y_test, final_preds)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='d',
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


KeyError: 'species'

Neural Network with PyTorch — Classifying Iris Flowers
Dataset: Iris Dataset (download the iris.csv)
Objective: Build and train a neural network using PyTorch to classify iris flowers into
one of three species.
1. Load & Preprocess the Data
o Load the dataset from the iris.csv file into a Pandas DataFrame.
o Encode the target column (species) as integer labels (0, 1, 2).
o Split the dataset into 80% training and 20% testing.
o Normalize the feature values (e.g., using StandardScaler).
2. Visualize the Data
o Create box plots for each feature (sepal length, sepal width, petal length, petal
width), separated by species to show the distribution and spread per class.
o Optional: Create a pairplot (scatterplot matrix) of all features, colored by
species.
3. Build the Neural Network
o Use PyTorch to define the following architecture:
• Input layer: 4 neurons (one for each feature)
• Hidden layer: 16 neurons with ReLU activation
• Output layer: 3 neurons
o Use CrossEntropyLoss as the loss function.
o Use an optimizer such as SGD or Adam.
4. Train the Model
o Train the model for 100 epochs.
o After every 10 epochs:
• Calculate and print training and test loss.
• Calculate and print training and test accuracy.
5. Create the Following Plots
o Plot 1: Training and test loss vs. epochs
o Plot 2: Training and test accuracy vs. epochs
o Plot 3: Final confusion matrix (visualized as a heatmap with class labels)